# Feature extractor (CNN with pretrained encoder layers) - optimization

Importing libraries

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np
from cnn_stacking import autoencoder,autoencoder1
from cnn_stacking import Cnn_Stacking
from dataloader_csv import CustomDatasetFromImages

Checking the availability of GPU 

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available(): 
    print("gpu available")

This function is used to save the state of the model

In [3]:
path_to_checkpoint='...//save_state_cnn_stack//'
check_name='checkpoint.pth.tar'
def save_checkpoint(state, is_best, filename,loss):
    """Save checkpoint if a new best is achieved"""
    if is_best:
        print ("=> Saving a new lowest loss : "+str(loss))
        torch.save(state, filename)  # save checkpoint

Following function takes 5 parameters. Purpose of this function is to train the encoder layer of autoencoder

In [4]:
def single_auto(model,_data,num_epochs,learning_rate,w_decay):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        loss_list=[]
        for images, labels in _data:

            images = images.to(device)

            outputs = model(images)

            loss = criterion(outputs,images)
            optimizer.zero_grad() 
            
            l1_reg = None
            for W in model.parameters():
                if l1_reg is None:
                    l1_reg = W.norm(p=1)
                else:
                    l1_reg = l1_reg + W.norm(p=1)
                                     
            loss=loss+l1_reg * w_decay
            
            loss.backward(retain_graph=True)
            
            optimizer.step()
            loss_list.append(loss.item())
        print('epoch [{}/{}], mean_loss:{:.4f}'.format(epoch + 1, num_epochs,np.mean(np.array(loss_list))))
    return model

The following function takes 6 parameters. This function first pass the data from pretrained encoder layer of an autoencoder and then pass it through the autoencoder1 for the pretraining of its encoder layer

In [5]:
def auto_to_auto(model_e,model_t,_data,num_epochs,learning_rate,w_decay):
    optimizer = torch.optim.Adam(model_t.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    for epoch in range(num_epochs):
        loss_list=[]
        for images, labels in _data:
          
            images = images.to(device)
            
            images=model_e.encoder(images)
         
            outputs = model_t(images)

            loss = criterion(outputs,images)
            
            optimizer.zero_grad() 
          
            
            l1_reg = None
            for W in model_t.parameters():
                if l1_reg is None:
                    l1_reg = W.norm(p=1)
                else:
                    l1_reg = l1_reg + W.norm(p=1)
                                     
            loss=loss+l1_reg * w_decay
            
            loss.backward(retain_graph=True)
            
            optimizer.step()
            loss_list.append(loss.item())
        print('epoch [{}/{}], mean_loss:{:.4f}'
              .format(epoch + 1, num_epochs,np.mean(np.array(loss_list))
        ))
    return model_t

Loading the train and validation data

In [6]:
TRAIN_DATA_PATH = "...//data_to_tra.csv"
VAL_DATA_PATH="...//data_to_val.csv"


train_data=CustomDatasetFromImages(TRAIN_DATA_PATH)
train_data_loader = data.DataLoader(train_data,batch_size=20,shuffle=True)


val_data=CustomDatasetFromImages(VAL_DATA_PATH)
val_data_loader = data.DataLoader(val_data,batch_size=20,shuffle=True)


Setting the parameters

In [7]:
num_epochs,learning_rate,weight_decay = 100,0.001,1e-5

Trainig the autoencoders

In [8]:
auto=single_auto(autoencoder().to(device).float(),train_data_loader,num_epochs,learning_rate,weight_decay)
auto1=auto_to_auto(auto,autoencoder1().to(device).float(),train_data_loader,num_epochs,learning_rate,weight_decay)

Creating the model (stacking the pretrained encoder layer for fine tuning purpose ), defining the loss function and optimizers 

In [10]:
model=Cnn_Stacking(auto,auto1).to(device).float()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-3)
criterion = nn.CrossEntropyLoss()

In this section, stacked encoder layers are fine tunned

In [11]:
total_step = len(train_data_loader)
lowest_loss=0.300

for epoch in range(num_epochs):

    for i, (images, labels) in enumerate(train_data_loader):
        
        images = images.to(device)
        labels = labels
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs,labels)

        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

        val_loss_list=[]
        for y,(val_images,val_labels) in enumerate(val_data_loader):

            val_images=val_images.to(device)

            val_labels=val_labels
            val_labels=val_labels.to(device)

            val_outputs=model(val_images)

            val_loss = criterion(val_outputs, val_labels)
            val_loss_list.append(val_loss.item())
        
        mean_loss= np.mean(np.array(val_loss_list))

        is_best= bool(mean_loss<lowest_loss)
        if(is_best):
            lowest_loss= min(mean_loss,lowest_loss)
            path= path_to_checkpoint+str(lowest_loss)+" "+str(epoch+1)+" "+check_name
            save_checkpoint({'epoch':epoch + 1,'state_dict': model.state_dict(),'lowest_loss': lowest_loss }, is_best,path,lowest_loss)
            is_best= False
        
        print("Epoch ["+str(epoch+1)+"/"+str(num_epochs)+"],Batch_no["+str(i+1)+"/"+str(total_step)+"] "+"Loss:"+str(format(loss.item(),'.4f'))+" Val_loss:"+str(format(mean_loss,'.4f')))